### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bile_Duct_Cancer/GSE107754'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Based on the dataset description, it contains whole human genome gene expression microarrays

# Check availability and identify the keys for the variables
trait_row = 2 if 'tissue: Bile duct cancer' in sample_characteristics[2] else None
age_row = None  # Age not explicitly mentioned
gender_row = 0 if 'gender: Male' in sample_characteristics[0] else None

# Define conversion functions
def convert_trait(value):
    if ':' in value:
        val = value.split(':')[1].strip().lower()
        if 'bile duct cancer' in val:
            return 1
        else:
            return 0
    return None

def convert_age(value):
    return None

def convert_gender(value):
    if ':' in value:
        val = value.split(':')[1].strip().lower()
        if val == 'male':
            return 1
        elif val == 'female':
            return 0
    return None

# Save cohort information
save_cohort_info('GSE107754', './preprocessed/Bile_Duct_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# If trait_row is not None, extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bile_Duct_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bile_Duct_Cancer/trait_data/GSE107754.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
